<a href="https://colab.research.google.com/github/Prakash-Repaka/Prakash-FMML/blob/main/FMML_M1L2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Machine Learning terms and metrics

FMML Module 1, Lab 2

In this lab, we will show a part of the ML pipeline by using the California Housing dataset. There are 20640 samples, each with 8 attributes like income of the block, age of the houses per district etc. The task is to predict the cost of the houses per district. We will use the scikit-learn library to load the data and perform some basic data preprocessing and model training. We will also show how to evaluate the model using some common metrics, split the data into training and testing sets, and use cross-validation to get a better estimate of the model's performance.

In [1]:
import numpy as np
from sklearn import datasets
import matplotlib.pyplot as plt

rng = np.random.default_rng(seed=42)

In [2]:
dataset = datasets.fetch_california_housing()
# Dataset description
print(dataset.DESCR)

.. _california_housing_dataset:

California Housing dataset
--------------------------

**Data Set Characteristics:**

    :Number of Instances: 20640

    :Number of Attributes: 8 numeric, predictive attributes and the target

    :Attribute Information:
        - MedInc        median income in block group
        - HouseAge      median house age in block group
        - AveRooms      average number of rooms per household
        - AveBedrms     average number of bedrooms per household
        - Population    block group population
        - AveOccup      average number of household members
        - Latitude      block group latitude
        - Longitude     block group longitude

    :Missing Attribute Values: None

This dataset was obtained from the StatLib repository.
https://www.dcc.fc.up.pt/~ltorgo/Regression/cal_housing.html

The target variable is the median house value for California districts,
expressed in hundreds of thousands of dollars ($100,000).

This dataset was derived

Given below are the list of target values. These correspond to the house value derived considering all the 8 input features and are continuous values. We should use regression models to predict these values but we will start with a simple classification model for the sake of simplicity. We need to just round off the values to the nearest integer and use a classification model to predict the house value.

In [3]:
print("Orignal target values:", dataset.target)

dataset.target = dataset.target.astype(int)

print("Target values after conversion:", dataset.target)
print("Input variables shape:", dataset.data.shape)
print("Output variables shape:", dataset.target.shape)

Orignal target values: [4.526 3.585 3.521 ... 0.923 0.847 0.894]
Target values after conversion: [4 3 3 ... 0 0 0]
Input variables shape: (20640, 8)
Output variables shape: (20640,)


The simplest model to use for classification is the K-Nearest Neighbors model. We will use this model to predict the house value with a K value of 1. We will also use the accuracy metric to evaluate the model.

In [4]:
def NN1(traindata, trainlabel, query):
    """
    This function takes in the training data, training labels and a query point
    and returns the predicted label for the query point using the nearest neighbour algorithm

    traindata: numpy array of shape (n,d) where n is the number of samples and d is the number of features
    trainlabel: numpy array of shape (n,) where n is the number of samples
    query: numpy array of shape (d,) where d is the number of features

    returns: the predicted label for the query point which is the label of the training data which is closest to the query point
    """
    diff = (
        traindata - query
    )  # find the difference between features. Numpy automatically takes care of the size here
    sq = diff * diff  # square the differences
    dist = sq.sum(1)  # add up the squares
    label = trainlabel[np.argmin(dist)]
    return label


def NN(traindata, trainlabel, testdata):
    """
    This function takes in the training data, training labels and test data
    and returns the predicted labels for the test data using the nearest neighbour algorithm

    traindata: numpy array of shape (n,d) where n is the number of samples and d is the number of features
    trainlabel: numpy array of shape (n,) where n is the number of samples
    testdata: numpy array of shape (m,d) where m is the number of test samples and d is the number of features

    returns: the predicted labels for the test data which is the label of the training data which is closest to each test point
    """
    predlabel = np.array([NN1(traindata, trainlabel, i) for i in testdata])
    return predlabel

We will also define a 'random classifier', which randomly allots labels to each sample

In [5]:
def RandomClassifier(traindata, trainlabel, testdata):
    """
    This function takes in the training data, training labels and test data
    and returns the predicted labels for the test data using the random classifier algorithm

    In reality, we don't need these arguments but we are passing them to keep the function signature consistent with other classifiers

    traindata: numpy array of shape (n,d) where n is the number of samples and d is the number of features
    trainlabel: numpy array of shape (n,) where n is the number of samples
    testdata: numpy array of shape (m,d) where m is the number of test samples and d is the number of features

    returns: the predicted labels for the test data which is a random label from the training data
    """

    classes = np.unique(trainlabel)
    rints = rng.integers(low=0, high=len(classes), size=len(testdata))
    predlabel = classes[rints]
    return predlabel

We need a metric to evaluate the performance of the model. Let us define a metric 'Accuracy' to see how good our learning algorithm is. Accuracy is the ratio of the number of correctly classified samples to the total number of samples. The higher the accuracy, the better the algorithm. We will use the accuracy metric to evaluate and compate the performance of the K-Nearest Neighbors model and the random classifier.

In [6]:
def Accuracy(gtlabel, predlabel):
    """
    This function takes in the ground-truth labels and predicted labels
    and returns the accuracy of the classifier

    gtlabel: numpy array of shape (n,) where n is the number of samples
    predlabel: numpy array of shape (n,) where n is the number of samples

    returns: the accuracy of the classifier which is the number of correct predictions divided by the total number of predictions
    """
    assert len(gtlabel) == len(
        predlabel
    ), "Length of the ground-truth labels and predicted labels should be the same"
    correct = (
        gtlabel == predlabel
    ).sum()  # count the number of times the groundtruth label is equal to the predicted label.
    return correct / len(gtlabel)

Let us make a function to split the dataset with the desired probability. We will use this function to split the dataset into training and testing sets. We will use the training set to train the model and the testing set to evaluate the model.

In [9]:
def split(data, label, percent):
    # generate a random number for each sample
    rnd = rng.random(len(label))
    split1 = rnd < percent
    split2 = rnd >= percent

    split1data = data[split1, :]
    split1label = label[split1]
    split2data = data[split2, :]
    split2label = label[split2]
    return split1data, split1label, split2data, split2label

We will reserve 20% of our dataset as the test set. We will not change this portion throughout our experiments

In [8]:
testdata, testlabel, alltraindata, alltrainlabel = split(
    dataset.data, dataset.target, 20 / 100
)
print("Number of test samples:", len(testlabel))
print("Number of train samples:", len(alltrainlabel))
print("Percent of test data:", len(testlabel) * 100 / len(dataset.target), "%")

Number of test samples: 4144
Number of train samples: 16496
Percent of test data: 20.07751937984496 %


## Experiments with splits

Let us reserve some of our train data as a validation set

In [10]:
traindata, trainlabel, valdata, vallabel = split(
    alltraindata, alltrainlabel, 75 / 100)

What is the accuracy of our classifiers on the train dataset?

In [11]:
trainpred = NN(traindata, trainlabel, traindata)
trainAccuracy = Accuracy(trainlabel, trainpred)
print("Training accuracy using nearest neighbour algorithm:", trainAccuracy*100, "%")

trainpred = RandomClassifier(traindata, trainlabel, traindata)
trainAccuracy = Accuracy(trainlabel, trainpred)
print("Training accuracy using random classifier: ", trainAccuracy*100, "%")

Training accuracy using nearest neighbour algorithm: 100.0 %
Training accuracy using random classifier:  16.4375808538163 %


For nearest neighbour, the train accuracy is always 1. The accuracy of the random classifier is close to 1/(number of classes) which is 0.1666 in our case. This is because the random classifier randomly assigns a label to each sample and the probability of assigning the correct label is 1/(number of classes). Let us predict the labels for our validation set and get the accuracy. This accuracy is a good estimate of the accuracy of our model on unseen data.

In [12]:
valpred = NN(traindata, trainlabel, valdata)
valAccuracy = Accuracy(vallabel, valpred)
print("Validation accuracy using nearest neighbour algorithm:", valAccuracy*100, "%")


valpred = RandomClassifier(traindata, trainlabel, valdata)
valAccuracy = Accuracy(vallabel, valpred)
print("Validation accuracy using random classifier:", valAccuracy*100, "%")

Validation accuracy using nearest neighbour algorithm: 34.10852713178294 %
Validation accuracy using random classifier: 16.884689922480618 %


Validation accuracy of nearest neighbour is considerably less than its train accuracy while the validation accuracy of random classifier is the same. However, the validation accuracy of nearest neighbour is twice that of the random classifier. Now let us try another random split and check the validation accuracy. We will see that the validation accuracy changes with the split. This is because the validation set is small and the accuracy is highly dependent on the samples in the validation set. We can get a better estimate of the accuracy by using cross-validation.

In [13]:
traindata, trainlabel, valdata, vallabel = split(
    alltraindata, alltrainlabel, 75 / 100)
valpred = NN(traindata, trainlabel, valdata)
valAccuracy = Accuracy(vallabel, valpred)
print("Validation accuracy using nearest neighbour algorithm:", valAccuracy*100, "%")

Validation accuracy using nearest neighbour algorithm: 34.048257372654156 %


You can run the above cell multiple times to try with different random splits.
We notice that the accuracy is different for each run, but close together.

Now let us compare it with the accuracy we get on the test dataset.

In [14]:
testpred = NN(alltraindata, alltrainlabel, testdata)
testAccuracy = Accuracy(testlabel, testpred)

print("Test accuracy:", testAccuracy*100, "%")

Test accuracy: 34.91795366795367 %


### Try it out for yourself and answer:
1. How is the accuracy of the validation set affected if we increase the percentage of validation set? What happens when we reduce it?
2. How does the size of the train and validation set affect how well we can predict the accuracy on the test set using the validation set?
3. What do you think is a good percentage to reserve for the validation set so that thest two factors are balanced?

Answer for both nearest neighbour and random classifier. You can note down the values for your experiments and plot a graph using  <a href=https://matplotlib.org/stable/gallery/lines_bars_and_markers/step_demo.html#sphx-glr-gallery-lines-bars-and-markers-step-demo-py>plt.plot<href>. Check also for extreme values for splits, like 99.9% or 0.1%

**1) Impact of Increasing or Decreasing the Percentage of Usage Listed:**

Percentage of the Validation Set: When you allocate a large portion of your data to the validation set, you have a more accurate estimate of how well your model will perform on unseen data. This is because the validation set will be more representative of the entire data distribution. However, this also means that you have limited data to train the model, which can lead to inefficiency during training due to not seeing many examples. On the other hand, a larger validation set provides better estimates of model performance but can degrade model training.

Reducing the Percentage of the Validation Set: Conversely, if you reduce the percentage of data reserved for validation, you have more data to shrink. This can help improve the model’s performance during training. However, the small validation set may not capture all the variability of the data, resulting in less reliable estimates of the model’s performance on unseen data. In particular, a small validation set yields imprecise estimates of model performance but allows for more robust training



**2)Size of Train and Validation Sets and Test Set Prediction Accuracy:**

Large Training Set: Generally a large training set allows the model to learn more effectively and capture patterns more efficiently, which can improve accuracy when predicting the test set. Too little training can lead to inadequacy, where the model fails to capture the underlying patterns in the data.

Validation Set Size: The validation set is used to tune hyperparameters and select the best model. A larger validation set provides better estimates of model performance, making it easier to select the best performing model. However, if the validation set is too small, the performance estimates may be unreliable, resulting in suboptimal model selection.

Effects on Test Set Prediction Accuracy: The validation set helps in selecting the model that is likely to generalize well for the test set. Thus, having a validation set of appropriate size increases the likelihood that the chosen model will perform well on the test set. The balance between sufficient training data and a reliable validation set is critical for optimal performance on the test set.




**3 ) A good percentage to save for a certification team:**

Typical practice: Typical practice is to reserve about 10% to 20% of data for validation. This percentage strikes a balance between obtaining enough data to adequately train the model and having a large enough validation set to obtain reliable performance estimates

Case Study: The ideal percentage may vary depending on the size of the overall data set and the specific problem. For very large data sets, even a small percentage (e.g. 5% to 10%) may be sufficient for the validation set. For smaller data sets, a larger fraction retention (e.g. 20%) may be required to obtain reliable estimates of model performance.

> Exercise: Try to implement a 3 nearest neighbour classifier and compare the accuracy of the 1 nearest neighbour classifier and the 3 nearest neighbour classifier on the test dataset. You can use the KNeighborsClassifier class from the scikit-learn library to implement the K-Nearest Neighbors model. You can set the number of neighbors using the n_neighbors parameter. You can also use the accuracy_score function from the scikit-learn library to calculate the accuracy of the model.

**Import Libraries**

In [15]:
import numpy as np
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score


**Load the data set**

In [16]:

data = load_iris()
X = data.data
y = data.target


**Split the Dataset**

In [18]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)


**Train and Evaluate 1-NN Classifier**

In [19]:
# Create the 1-NN classifier
knn_1 = KNeighborsClassifier(n_neighbors=1)

# Train the model
knn_1.fit(X_train, y_train)

# Predict on the test set
y_pred_1 = knn_1.predict(X_test)

# Calculate accuracy
accuracy_1 = accuracy_score(y_test, y_pred_1)
print(f"Accuracy of 1-NN: {accuracy_1:.4f}")


Accuracy of 1-NN: 1.0000


**Train and Evaluate 3-NN Classifier**

In [20]:
# Create the 3-NN classifier
knn_3 = KNeighborsClassifier(n_neighbors=3)

# Train the model
knn_3.fit(X_train, y_train)

# Predict on the test set
y_pred_3 = knn_3.predict(X_test)

# Calculate accuracy
accuracy_3 = accuracy_score(y_test, y_pred_3)
print(f"Accuracy of 3-NN: {accuracy_3:.4f}")


Accuracy of 3-NN: 1.0000


In [ ]:
def AverageAccuracy(alldata, alllabel, splitpercent, iterations, classifier=NN):
    """
    This function takes in the data, labels, split percentage, number of iterations and classifier function
    and returns the average accuracy of the classifier

    alldata: numpy array of shape (n,d) where n is the number of samples and d is the number of features
    alllabel: numpy array of shape (n,) where n is the number of samples
    splitpercent: float which is the percentage of data to be used for training
    iterations: int which is the number of iterations to run the classifier
    classifier: function which is the classifier function to be used

    returns: the average accuracy of the classifier
    """
    accuracy = 0
    for ii in range(iterations):
        traindata, trainlabel, valdata, vallabel = split(
            alldata, alllabel, splitpercent
        )
        valpred = classifier(traindata, trainlabel, valdata)
        accuracy += Accuracy(vallabel, valpred)
    return accuracy / iterations  # average of all accuracies

In [ ]:
avg_acc = AverageAccuracy(alltraindata, alltrainlabel, 75 / 100, 10, classifier=NN)
print("Average validation accuracy:", avg_acc*100, "%")
testpred = NN(alltraindata, alltrainlabel, testdata)

print("Test accuracy:", Accuracy(testlabel, testpred)*100, "%")

Average validation accuracy: 33.58463539517022 %
Test accuracy: 34.91795366795367 %


This is a very simple way of doing cross-validation. There are many well-known algorithms for cross-validation, like k-fold cross-validation, leave-one-out etc. This will be covered in detail in a later module. For more information about cross-validation, check <a href=https://en.wikipedia.org/wiki/Cross-validation_(statistics)>Cross-validatioin (Wikipedia)</a>

### Questions
1. Does averaging the validation accuracy across multiple splits give more consistent results?
2. Does it give more accurate estimate of test accuracy?
3. What is the effect of the number of iterations on the estimate? Do we get a better estimate with higher iterations?
4. Consider the results you got for the previous questions. Can we deal with a very small train dataset or validation dataset by increasing the iterations?


**1 Does Averaging the Validation Accuracy Across Multiple Splits Give More Consistent Results?**

Yes, averaging validation accuracy across multiple splits tends to yield consistent and reliable estimates of model performance. This approach, called cross-validation, involves partitioning the data into several subsets (folds), training the model on some of these folds and validating it on the remaining folds. By binding the results of each fold, you reduce the impact of any single data split and obtain a more robust estimate of the performance of the model.

Interest:

Reduced variance: Reduces the variance associated with a single validation set, since the performance metrics are averaged over multiple partitions.
More Reliable Estimates: Provides more reliable estimates of how the model will perform on unobserved data, when the model is tested on a wide variety of subgroups.

**2. Does It Give a More Accurate Estimate of Test Accuracy?**


The average validation accuracy across multiple partitions yields a better estimate of the model’s performance on new data compared to a single validation partition. However, it does not directly estimate the accuracy of the test but reflects the general performance of the model. Experimental accuracy is measured on the most unique trials that were not detected by the model during training or validation.

Highlights:

Cross-validation estimates: Provides a good estimate of how well the model can perform in an independent test set.
Test set accuracy: The actual performance on the test set is still a sure indicator of how well the model generalizes, if the test set is representative and independent of the training and validation sets

**3. What Is the Effect of the Number of Iterations on the Estimate? Do We Get a Better Estimate with Higher Iterations?**


Increasing the number of iterations (or folds in cross-validation) generally results in more reliable estimates of model performance. Here’s why:

Frequency (Higher Iterations): When multiple folds are used (e.g., 10-fold cross-validation), each model is trained and validated on a slightly different portion of the data, and the results are averaged. This reduces the variance in estimates of model performance and provides a more accurate measure of how the model performs in different subsets of the data.
Communication: However, increasing the number of folds increases the computational cost and time, as the model must be trained and validated repeatedly.
Best Practice: Common choices are 5-fold or 10-fold cross-validation, balancing accuracy and statistical efficiency.

**4. Can We Deal with a Very Small Train Dataset or Validation Dataset by Increasing the Iterations?**

Increasing the number of iterations (or folds) in cross-validation can help to some extent when dealing with small datasets by maximizing the use of available data. Here's how it looks:

Data Efficiency: For very small datasets, each model gets both training and validation in different iterations. This can be useful because it ensures that each sample contributes to the performance analysis repeatedly.
Increased stability: Cross-validation with multiple folds (e.g., leave-one-out cross-validation) helps to obtain robust estimates of model performance by using almost all data for training and evaluation a are performed on only one sample at a time.
However, increasing the number of folds cannot fully compensate for the limitations of a small dataset. Even with multiple iterations, the results may still be highly unreliable due to the limitations of small data sets, such as high variability and risks of overfitting. In such a case, measures such as data enrichment, or more data acquisition, if possible, may be necessary to improve model performance and reliability.

> Exercise: How does the accuracy of the 3 nearest neighbour classifier change with the number of splits? How is it affected by the split size? Compare the results with the 1 nearest neighbour classifier.

The accuracy of the 3 nearest neighbour classifier can be affected by the number of splits and the split size. Generally, as the number of splits increases, the accuracy of the classifier may improve, but it can also lead to overfitting. This is because the classifier is trying to fit the data too closely, which can result in poor generalization to new, unseen data.

On the other hand, the split size can also affect the accuracy of the classifier. If the split size is too small, the classifier may not have enough data to make accurate predictions. If the split size is too large, the classifier may be biased towards the majority class.

Here is an example of how the accuracy of the 3 nearest neighbour classifier can change with the number of splits and split size:



In [21]:
import numpy as np
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split

# Generate some sample data
X = np.random.rand(100, 2)
y = np.random.randint(0, 2, 100)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Create a 3 nearest neighbour classifier
knn = KNeighborsClassifier(n_neighbors=3)

# Train the classifier on the training data
knn.fit(X_train, y_train)

# Make predictions on the testing data
y_pred = knn.predict(X_test)

# Calculate the accuracy of the classifier
accuracy = np.mean(y_pred == y_test)
print("Accuracy:", accuracy)

# Now, let's see how the accuracy changes with the number of splits
for i in range(2, 11):
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=i)
    knn.fit(X_train, y_train)
    y_pred = knn.predict(X_test)
    accuracy = np.mean(y_pred == y_test)
    print("Number of splits:", i, "Accuracy:", accuracy)

# Now, let's see how the accuracy changes with the split size
for i in [0.1, 0.2, 0.3, 0.4, 0.5]:
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=i, random_state=42)
    knn.fit(X_train, y_train)
    y_pred = knn.predict(X_test)
    accuracy = np.mean(y_pred == y_test)
    print("Split size:", i, "Accuracy:", accuracy)

Accuracy: 0.45
Number of splits: 2 Accuracy: 0.5
Number of splits: 3 Accuracy: 0.4
Number of splits: 4 Accuracy: 0.5
Number of splits: 5 Accuracy: 0.6
Number of splits: 6 Accuracy: 0.35
Number of splits: 7 Accuracy: 0.5
Number of splits: 8 Accuracy: 0.5
Number of splits: 9 Accuracy: 0.65
Number of splits: 10 Accuracy: 0.4
Split size: 0.1 Accuracy: 0.4
Split size: 0.2 Accuracy: 0.45
Split size: 0.3 Accuracy: 0.4666666666666667
Split size: 0.4 Accuracy: 0.525
Split size: 0.5 Accuracy: 0.44
